In [1]:
# 常用的引用和创建对象实体
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)

# 1, Motivation

# 2, Creating Pair RDDs

Always creat Pair RDDs by using RDD.map()

In [2]:
# Examples
lines = sc.parallelize(["hello world","hi Worriors","for the horde, bool and lighting!!!"])
# create a pair RDD by using the first word as the KEY
pairs = lines.map(lambda x: (x.split(" ")[0],x))
print(pairs.collect())

[('hello', 'hello world'), ('hi', 'hi Worriors'), ('for', 'for the horde, bool and lighting!!!')]


# 3, Transformations on Pair RDDs

Pairs RDDs are allowed to use all the transformations available to standard RDDs. Since Pair RDDs contains tuples, we need to pass functions that operate on tuples rather than on individual elements.

Pair RDDs are also still RDDs, and thus support the same functions as RDDs.

In [3]:
# filter the second elements of lines
result = pairs.filter(lambda keyValue: len(keyValue[1]) < 20)
print(result.collect())

[('hello', 'hello world'), ('hi', 'hi Worriors')]


## 3.1 Aggregations

### Functions:
1. reduceByKey():**similar with reduce()**, runs several parallel reduce operations, one for each key in the dataset, where each operation combines values that have the same key. It returns a new RDD consisting of each key and the reduced value for that key.
2. foldByKey(): **similar with fold()**
3. mapValues(): Usecase in the picture below

<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/c0ba7cc4ceb1651a33bc65f0f3c738d68abbf25f/ipynbFiles/Materials/Usecase1.jpg" />

In [25]:
# examples of picture
# How create a Pais RDD
word = [('panda',0),('pink',3),('pirate',3),('panda',1),('pink',4)] # List insided with tuples
Words = sc.parallelize(word)
print(Words.keys().collect())
print(Words.values().collect())
new_words = Words.mapValues(lambda x:(x,1))
print(new_words.values().collect())
NewWords = new_words.reduceByKey(lambda x,y:(x[0] + y[0], x[1] + y[1]))
print(NewWords.values().collect())
print(NewWords.collect())


['panda', 'pink', 'pirate', 'panda', 'pink']
[0, 3, 3, 1, 4]
[(0, 1), (3, 1), (3, 1), (1, 1), (4, 1)]
[(1, 2), (7, 2), (3, 1)]
[('panda', (1, 2)), ('pink', (7, 2)), ('pirate', (3, 1))]


In [38]:
# Example of WordCount of a file(total number of each word)
File_RDD = sc.textFile("/home/colin/bashrc")
Txt = File_RDD.flatMap(lambda x:x.split(" ")) # this step is important!!!
print(Txt.take(10))
result = Txt.map(lambda x:(x,1)).reduceByKey(lambda x,y: x + y)
print(result.take(10))
Res = Txt.countByValue() # countByValue() count each word, and return a collections.defaultdict
print(type(Res))
print(Res[0])


['', '#', 'see', '/usr/share/doc/bash/examples/startup-files', '(in', 'the', 'package', 'bash-doc)', '#', 'for']
[('', 598), ('#', 73), ('see', 3), ('/usr/share/doc/bash/examples/startup-files', 1), ('(in', 1), ('the', 27), ('package', 1), ('bash-doc)', 1), ('for', 9), ('examples', 1)]
<class 'collections.defaultdict'>
0
